This function computes the tensorlines of the eigenvector-field $ \mathbf{e}_i(\mathbf{x}) $. Due to the unavoidable discontinuities of the eigenvector-field, the eigenvectors have to be re-oriented on the fly before each integration step.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| eig | array (Ny, Nx) | eigenvalue-field |
| vector_field | array (Ny, Nx, 2) | eigenvector-field |
| min_distance | float | minimum distance between local maxima |
| max_length | float | maximum length of tensorline |
| step_size | float | integration step size |
| hyperbolicity | float | stop integration of tensorlines if rate of attraction/repulsion < hyperbolicity|
| n_iterations | float | stop integration of tensorlines if number of iterations > n_iterations|

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# import math tools
from math import sqrt

# import function to compute local maxima
from ipynb.fs.defs.loc_max import _loc_max

# import function to orient vectorfield
from ipynb.fs.defs.orient_vectorfield import _orient_vectorfield

# import function to integrate tensorlines
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines

# Import function to compute eigenvalues/eigenvectors
from ipynb.fs.defs.eigen import eigen

# Import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

In [3]:
def _tensorlines(X, Y, eig, vector_field, min_distance, max_length, step_size, hyperbolicity = 0, n_iterations = 10**4):
    '''
    Wrapper for RK4_tensorlines(). Integrates the tensorlines given an eigenvector/eigenvalue field. The integration stops
    whenever a threshold value is reached (max_length/hyperbolicity/counter_threshold).
    
    Parameters:
        X:                 array (Ny, Nx), X-meshgrid (of complete data domain).
        Y:                 array (Ny, Nx), Y-meshgrid (of complete data domain).
        eig:               array (Ny, Nx), eigenvalue field.
        vector_field:      array (Ny, Nx, 2), eigenvector field.
        min_distance:      float, minimum distance between local extrema in the eigenvalue field.
        max_length:        float, maximum length of tensorlines.
        step_size:         float, step size used for integration. This value is kept constant.
        hyperbolicity:     float, threshold on hyperbolicity value. 
                           If point on tensorline has hyperbolicity lower than this threshold, than integration is stop.
        n_iterations: int, threshold on number of iterations.
    
    Returns:
        tensorlines:       list, each element in the list contains an array specifying the x/y-coordinates of the tensorlines.
    '''
    
    
    # Find local extrema of eigfield
    peak_x, peak_y, peak_field = _loc_max(min_distance, X, Y, eig)[2:]
    
    # defined domain
    defined_domain = np.isfinite(eig).astype(int) # array (Ny, Nx)
    
    # set nan values of eig to zero for gridded interpolation
    eig = np.nan_to_num(eig, 0) # array (Ny, Nx)
        
    # Interpolate Eigenvalue field
    interp_eig = RBS(Y[:,0], X[0,:], eig, kx = 1, ky = 1) # RectangularBivariateSpline object
    
    # Define list of tensorlines (back/forward)
    tensorlines = [[], []]
    
    # Iterate over all local maxima
    for i in range(len(peak_x)):
        
        tensorlines_forw = [[], []]
        tensorlines_back = [[], []]
        
        # Local maxima point
        x = np.array([peak_x[i], peak_y[i]]) # array (2,)
        
        if peak_field[i] > hyperbolicity:
            # Boolean forward Iteration
            bool_forward, bool_backward = True, True # bool, bool
        
        else:
            bool_forward, bool_backward = False, False # bool, bool
        
        # Start integration only if local maxima is not close than 'max_distance' to shrinkline
        if bool_forward and bool_backward:
            
            # Starting point of integration
            x_forward = x # array (2,)
            x_backward = x # array (2,)
            
            # Append starting point to list containing positions of forward shrinklines
            for ii in range(2):
                tensorlines_forw[ii].append(x[ii])
                
            # Check orientation of vector-field and rieorient if needed.
            vx, vy = _orient_vectorfield(X, Y, x, vector_field)
            
            # Initial vector orientation
            x_prime_forward = np.array([vx, vy]) # array (2,)
            x_prime_backward = -np.array([vx, vy]) # array (2,)
            
            # Initial distance
            dist = 0
            
            # set counter for number of iterations
            counter = 0
            
            # Start Integration with dummy variable 's_array'
            while bool_forward or bool_backward:
                
                # If number of iterations exceeds limit --> stop integration
                if counter > n_iterations: 
                    bool_forward = False
                    bool_backward = False
                    
                # Integrate only if 'x_prime_forward' is defined and 'bool_forward == True'
                if bool_forward and x_prime_forward is not None:
                    
                    eig_old = abs(interp_eig(x_forward[1], x_forward[0])[0][0])
                    
                    # RK4 integration for tensorline
                    x_forward, x_prime_forward = _RK4_tensorlines(X, Y, defined_domain, x_forward, x_prime_forward, step_size, vector_field, interp_eig) 
                        
                    if x_forward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_forward[0]**2+x_prime_forward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_forw[ii].append(x_forward[ii])
                                
                        else:
                            
                            bool_forward = False
                            
                        eig_new = abs(interp_eig(x_forward[1], x_forward[0])[0][0])
                            
                        if eig_new < hyperbolicity or eig_new > eig_old:
                            
                            bool_forward = False
                    else:
                        
                        bool_forward = False
                
                # Integrate only if 'x_prime_backward' is defined and 'bool_backward == True'
                if bool_backward and x_prime_backward is not None:
                    
                    eig_old = np.abs(interp_eig(x_backward[1], x_backward[0])[0][0])
                    
                    # RK4 integration for tensorline
                    x_backward, x_prime_backward = _RK4_tensorlines(X, Y, defined_domain, x_backward, x_prime_backward, step_size, vector_field, interp_eig) 
                    
                    if x_backward is not None:
                        
                        # Compute length of tensorline
                        dist += sqrt(x_prime_backward[0]**2+x_prime_backward[1]**2)*step_size
                        
                        # If distance is below length of tensorline --> append point to tensorline
                        if dist < max_length:
                            for ii in range(2):
                                tensorlines_back[ii].append(x_backward[ii])
                    
                        else:
                            
                            bool_backward = False
                            
                        eig_new = abs(interp_eig(x_backward[1], x_backward[0])[0][0])
                        
                        if eig_new < hyperbolicity or eig_new > eig_old:
                            
                            bool_backward = False
                    
                    else:
                        
                        bool_backward = False
                  
                counter += 1
                    
                        
            # Append backward and forward shrinkline
            for ii in range(2):
                tensorlines[ii].append(np.append(np.flip(tensorlines_back[ii]), tensorlines_forw[ii]))
    
    return tensorlines